In [508]:
import pandas as pd
from bs4 import BeautifulSoup
import re

In [509]:
def job_rating(card):
    try:
        job_rating=card.find('span','ratingsDisplay').a.text
    except AttributeError:
        job_rating=''
    return job_rating

def job_salary(card):
    try:
        job_salary=card.find(class_='salary-snippet-container').text
        # job_salary=card.find('div','metadata salary-snippet-container').text
    except AttributeError:
        job_salary=''
    return job_salary

def get_job_id(soup):
    jid_list=[]
    find_jid = re.compile(r'(jobKeysWithInfo\[([a-zA-Z0-9]{16})\])')
    script = soup.find("script", text=lambda text: text and "var jobKeysWithInfo" in text).text
    for row in script.split('\n'):
            if row.startswith('jobKeysWithInfo'):
                jid = find_jid.search(row.replace("'", ""))    
                if jid:
                    jid = find_jid.search(row.replace("'", "")).group(2)
                    jid_list.append(jid)
    return jid_list

In [506]:
#Set the structual of dataframe
def df_basic(raw):
    df = pd.read_csv(raw)
    df_basic = pd.DataFrame(columns=("job_title","company_name","location",'salary','rating',"job_id"))
    for i in df['HTML']:
        soup=BeautifulSoup(i,'html.parser')
        script = soup.find("script", text=lambda text: text and "var jobKeysWithInfo" in text).text
        """ get list of id and list of "company_name","location",'salary','rating',"job_id" """
        id_list=get_job_id(soup)
        cards=soup.find_all('div','job_seen_beacon')
        result=[[card.h2.text,card.find("div",'heading6').contents[0].text,card.find('div','companyLocation').text,
                job_salary(card),job_rating(card)] for card in cards]
        """join two list """
        [result[i].append(id_list[i]) for i in range(len(result))]
        for r in range(len(result)):
            df_basic=df_basic.append(pd.Series(result[r], index = ["job_title","company_name","location",'salary','rating',"job_id"]), ignore_index=True)
        """return into dataframe with order"""
        df_basic=df_basic[["job_id","job_title","company_name","location",'salary','rating']]
        '''output result.csv'''
        # df_basic.to_csv('result.csv')
    return df_basic


In [510]:
df_basic("Data Analyst_New York State_30.csv")
